In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-29 01:20:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.07MB/s    in 0.2s    

2021-03-29 01:20:36 (6.07 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
#Import columns function
from pyspark.sql.functions import col

In [6]:
#Filter data to for when total votes >= 20
popular_products_df = df.filter(col("total_votes")>=20)
popular_products_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [9]:
#Filter data further to only show rows when percentage of helpful votes to total votes is 50% or higher
helpful_reviews_df = popular_products_df.filter(col("helpful_votes")/col("total_votes")>=0.5)
helpful_reviews_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [30]:
#Filter data to show only paid vine program
paid_df = helpful_reviews_df.filter(col("vine")=="Y")
paid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44759540|R3A71VR1JZD8WF|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          2|           27|         30|   Y|                N|Great idea, not s...|I do not hate thi...| 2015-08-29|
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|    

In [11]:
#Filter data to show only unpaid vine program
unpaid_df = helpful_reviews_df.filter(col("vine")=="N")
unpaid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [16]:
#Determine total number of paid reviews
paid_df.count()


170

In [17]:
#Determine total number of 5 stars paid reviews
paid_df.filter(col("star_rating")==5).count()


65

In [22]:
#Determine total percentage of 5 stars paid reviews
(paid_df.filter(col("star_rating")==5).count()/paid_df.count())*100

38.23529411764706

In [23]:
#Determine total number of unpaid reviews
unpaid_df.count()

37840

In [24]:
#Determine total number of 5 stars unpaid reviews
unpaid_df.filter(col("star_rating")==5).count()

20612

In [26]:
#Determine total percentage of 5 stars unpaid reviews
(unpaid_df.filter(col("star_rating")==5).count()/unpaid_df.count())*100

54.471458773784356